# 初始化
当进入此界面时, 项目会自动初始化, 包括从云端加载数据集, 可能会需要1分钟左右完成. 当数据集加载后, 可使用下方命令检查数据集是否已经被加载

In [ ]:
# 查看当前挂载的数据集目录 
!ls /home/aistudio/data/data112
# !ls /home/aistudio/data/data194

# 预处理
将数据集压缩包解压, 由于该数据集体积为1.8GB, 因而解压较慢. 约15-20分钟. 

In [ ]:
import tarfile
import os

tar = tarfile.open("/home/aistudio/data/data112/baidu_star_2018_train_stage1.tar")
tar.extractall("/home/aistudio/data/data112/")
tar.close()


# 验证是否解压成功
解压后的图片文件应该有3625张, 如不符, 请清空后重新解压. 长时间不能成功, 请联系管理员

In [ ]:
!ls /home/aistudio/data/data112/baidu_star_2018/image/stage1/train

In [ ]:
!ls /home/aistudio/data/data112/baidu_star_2018/annotation/

# 可以使用pip自行安装需要的package库

In [ ]:
!pip install lmdb -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install sklearn
!pip install git
!pip install matlab

# 请在下方添加您的处理代码

In [ ]:
!git clone https://github.com/val-iisc/crowd-counting-scnn.git

# create density map

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2

img_fold = '/home/aistudio/data/data112/baidu_star_2018/image/'
# 读取数据
with open('/home/aistudio/data/data112/baidu_star_2018/annotation/annotation_train_stage1.json', 'r') as f:
    ann_train = json.load(f)


In [ ]:
def gaussian_kernel(kernel_x = 15,kernel_y = 15,sigma = 4.0):
    kx = cv2.getGaussianKernel(kernel_x,sigma)
    ky = cv2.getGaussianKernel(kernel_y,sigma)
    H = np.multiply(kx,np.transpose(ky)) 
    return np.transpose(H)

def rbg2gray(img):
    weight = [0.2125, 0.7154, 0.0721]
    return np.dot(img, weight)

def density_map(im, points):
    w, h, c = im.shape

    if c == 3:
        im = rbg2gray(im)

    im_density = np.zeros(im.shape)

    # 长度为0返回
    if len(points) == 0:
        return im_density

    # 长度为1
    if len(points) == 1:
        x1 = max(1, min(w, round(points[0, 0])))
        y1 = max(1, min(h, round(points[0, 1])))

        im_density[y1, x1] = 255
        return im_density

    for j in range(len(points)):
        f_sz = 15
        sigma = 4.0

        # 获取一个高斯函数矩阵
        H = gaussian_kernel(kernel_x = f_sz,kernel_y = f_sz,sigma = sigma)

        x = min(w, max(1, abs(int(math.floor(points[j, 0])))))
        y = min(h, max(1, abs(int(math.floor(points[j, 1])))))

        if x > w | y > h:
            continue

        radius = int(math.floor(f_sz / 2))

        x1, x2 = x - radius, x + radius
        y1, y2 = y - radius, y + radius

        dfx1, dfy1, dfx2, dfy2 = 0, 0, 0, 0

        change_H = False

        if x1 < 1:
            dfx1 = abs(x1) + 1
            x1 = 1
            change_H = True

        if y1 < 1:
            dfy1 = abs(y1) + 1
            y1 = 1
            change_H = True

        if x2 > w:
            dfx2 = x2 - w
            x2 = w
            change_H = True

        if y2 > h:
            dfy2 = y2 - h
            y2 = h
            change_H = True

        x1h = 1 + dfx1
        y1h = 1 + dfy1
        x2h = f_sz - dfx2
        y2h = f_sz - dfy2

        if change_H == True:
            kx = y2h - y1h + 1
            ky = x2h - x1h + 1
            H = gaussian_kernel(kernel_x = ky,kernel_y = kx,sigma = sigma)

        im_density[y1: y2 + 1, x1: x2 + 1] = im_density[y1: y2 + 1, x1: x2 + 1] + H

    return im_density

In [ ]:
# print (gaussian_kernel(3,2,4)) # 测试高斯核

In [ ]:
ann = ann_train.get('annotations')
length = len(ann)
print('totle size is %d '%length)

# 读取图片
I = mpimg.imread('/home/aistudio/data/data112/baidu_star_2018/image/'+ann[1].get('name'))
print(I.shape)
plt.imshow(I)
locs    = ann[0].get('annotation');

x = locs[1].get('x')
y = locs[1].get('y')
point = np.array([x,y])
point


In [ ]:

for j in range(len(locs)):
    x = locs[j].get('x')
    y = locs[j].get('y')
    point = np.array([x,y])
    if j == 0:
        gt4map  = point
    else:
        gt4map = np.vstack((gt4map,point))

print(gt4map)

print(I[0,0,0])

In [ ]:

# Main Loop
for i in range(length):
    print('iter : %d '%(i))
    ann_i = ann[i]
    cnt = ann_i.get('num')
    
    I = mpimg.imread( img_fold + ann_i.get('name'))
    locs = ann_i.get('annotation');
    
    count   = 1;
    gt4map  = np.array([]);
    
    if i ==3:
        break
    
    if ann_i.get('type') == 'dot':
        for j in range(len(locs)):
            x = locs[j].get('x')
            y = locs[j].get('y')
            point = np.array([x,y])
            if j == 0:
                gt4map  = np.array([x,y])
            else:
                gt4map = np.vstack([gt4map,point])
        
    if  ann_i.get('type') == 'bbox':
        continue
        
    file_name   = ann_i.get('name');
    count       = 1; # 标记patch
    
    h, w, c = I.shape
    
    # create density map
    d_map = density_map(I, gt4map)
    plt.imshow(I)
    plt.show()
    plt.imshow(d_map)
    plt.show()
    
    # 验证总和
    print('gt cnt: %d'%cnt)
    print('precoss cnt : %f'%np.sum(np.sum(d_map)))
    
    # d_map = create_density(gt / 4.0, d_map_h, d_map_w)

    p_h = math.floor(I.shape[0] / 3.0)
    p_w = math.floor(I.shape[1] / 3.0)
    d_map_ph = p_h
    d_map_pw = p_w
    
    # create non-overlapping patches of images and density maps
    py = 0
    py2 = 0
    for j in range(3):
        px = 0
        px2 = 0
        for k in range(3):
            final_image = np.array(I[int(py): int(py + p_h), int(px): int(px + p_w), :], dtype="double")
            final_gt = np.array(d_map[int(py2): int(py2 + d_map_ph ), int(px2): int( px2 + d_map_pw )], dtype="double")
            
            plt.subplot(121)
            plt.imshow(final_image)
            plt.subplot(122)
            plt.imshow(final_gt)
            plt.show()
            
            px = px + p_w + 1
            px2 = px2 + d_map_pw + 1

            # if final_image.shape[2] < 3:
            #     final_image = repmat(final_image, [1, 1, 3])
            
            img_name = '%spatch/%s_%d.png'%(img_fold,ann_i.get('name')[:-4],count)
            gt_name = '%sgt/%s_%d.png'%(img_fold,ann_i.get('name')[:-4],count)
            plt.imsave(img_name,final_image)
            plt.imsave(gt_name,final_gt)
            
            count = count + 1
        py = py + p_h 
        py2 = py2 + d_map_ph 
